# select random 20 sample from dermnet
### prepare pesudo test

In [8]:
import os
import random

# === 配置 ===
train_root = "/data_lg/keru/project/part2/DermNet/train"
output_txt = "/data_lg/keru/project/part2/DermNet/pseudo_eval/pseudo_test_list.txt"

num_total = 20      # 总数
num_acne = 10        # Acne 样本数
num_non_acne = num_total - num_acne

# === 找到所有子文件夹 ===
all_classes = [
    d for d in os.listdir(train_root)
    if os.path.isdir(os.path.join(train_root, d))
]

# Acne 类和 Non-Acne 类分开
acne_folders = [d for d in all_classes if "acne" in d.lower()]
non_acne_folders = [d for d in all_classes if d not in acne_folders]

assert acne_folders, "没有找到包含 'acne' 的文件夹！"

# === 随机选 Acne 样本 ===
acne_samples = []
for folder in acne_folders:
    folder_path = os.path.join(train_root, folder)
    imgs = [os.path.join(folder_path, f) for f in os.listdir(folder_path)
            if f.lower().endswith((".jpg", ".jpeg", ".png"))]
    acne_samples.extend(imgs)
pseudo_acne = random.sample(acne_samples, num_acne)

# === 随机选 Non-Acne 样本（保证尽量不同文件夹）===
pseudo_non_acne = []
selected_folders = random.sample(non_acne_folders, min(num_non_acne, len(non_acne_folders)))
for folder in selected_folders:
    folder_path = os.path.join(train_root, folder)
    imgs = [os.path.join(folder_path, f) for f in os.listdir(folder_path)
            if f.lower().endswith((".jpg", ".jpeg", ".png"))]
    if imgs:
        pseudo_non_acne.append(random.choice(imgs))

# 若文件夹不够，继续补
while len(pseudo_non_acne) < num_non_acne:
    folder = random.choice(non_acne_folders)
    folder_path = os.path.join(train_root, folder)
    imgs = [os.path.join(folder_path, f) for f in os.listdir(folder_path)
            if f.lower().endswith((".jpg", ".jpeg", ".png"))]
    if imgs:
        pseudo_non_acne.append(random.choice(imgs))

# === 合并 & 保存 txt ===
with open(output_txt, "w") as f:
    for p in pseudo_acne:
        f.write(f"{p}\t1\n")
    for p in pseudo_non_acne:
        f.write(f"{p}\t0\n")

print(f"Pseudo test list saved to {output_txt}")
print(f"Acne: {len(pseudo_acne)}, Non-Acne: {len(pseudo_non_acne)}")


Pseudo test list saved to /data_lg/keru/project/part2/DermNet/pseudo_eval/pseudo_test_list.txt
Acne: 10, Non-Acne: 10


### test_pseudo

In [9]:
import os
import torch
from torchvision import transforms, models
from torch import nn
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from tqdm import tqdm
from PIL import Image
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
import json

# === 配置 ===
pseudo_txt = "/data_lg/keru/project/part2/DermNet/pseudo_eval/pseudo_test_list.txt"
model_path = "/data_lg/keru/project/part2/outputs/best_resnet50.pt"
gradcam_dir = "/data_lg/keru/project/part2/DermNet/pseudo_eval/gradcam_pseudo"

os.makedirs(gradcam_dir, exist_ok=True)

# === 加载模型 ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet50(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, 2)
model.load_state_dict(torch.load(model_path))
model = model.to(device)
model.eval()

# === 预处理 ===
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# === 读取伪测试集列表 ===
samples = []
labels = []
with open(pseudo_txt) as f:
    for line in f:
        path, lbl = line.strip().split("\t")
        samples.append(path)
        labels.append(int(lbl))

# === Grad-CAM 设置 ===
target_layer = model.layer4[-1]
cam = GradCAM(model=model, target_layers=[target_layer])

# === 推理 ===
all_preds = []
all_probs = []

for img_path, true_label in tqdm(zip(samples, labels), total=len(samples)):
    img = Image.open(img_path).convert("RGB")
    input_tensor = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(input_tensor)
        prob = torch.softmax(output, dim=1)[0, 1].item()
        pred = output.argmax(dim=1).cpu().item()

    all_preds.append(pred)
    all_probs.append(prob)

    # === Grad-CAM 可视化 ===
    grayscale_cam = cam(input_tensor=input_tensor, targets=[ClassifierOutputTarget(1)])[0, :]
    rgb_img = transform(img).permute(1, 2, 0).cpu().numpy()
    rgb_img = (rgb_img - rgb_img.min()) / (rgb_img.max() - rgb_img.min())
    cam_image = show_cam_on_image(rgb_img, grayscale_cam, use_rgb=True)

    out_name = os.path.basename(img_path).split(".")[0] + "_gradcam.jpg"
    out_path = os.path.join(gradcam_dir, out_name)
    Image.fromarray(cam_image).save(out_path)

print(f"✅ Grad-CAM saved to {gradcam_dir}")

# === 计算指标 ===
acc = accuracy_score(labels, all_preds)
prec = precision_score(labels, all_preds)
rec = recall_score(labels, all_preds)
f1 = f1_score(labels, all_preds)
try:
    auc = roc_auc_score(labels, all_probs)
except:
    auc = float('nan')

print(f"Accuracy: {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall: {rec:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"AUROC: {auc:.4f}")

import json
import os
# === 保存详细预测信息和整体评估指标 ===
results = []
for path, true_label, pred, prob in zip(samples, labels, all_preds, all_probs):
    results.append({
        "image": path,
        "true_label": true_label,
        "pred_label": pred,
        "probability": round(prob, 4),
        "correct": int(pred == true_label)
    })

metrics = {
    "results": results,
    "summary": {
        "accuracy": round(acc, 4),
        "precision": round(prec, 4),
        "recall": round(rec, 4),
        "f1_score": round(f1, 4),
        "auroc": round(auc, 4)
    }
}

save_path = "/data_lg/keru/project/part2/DermNet/pseudo_eval/pseudo_eval_metrics.json"
os.makedirs(os.path.dirname(save_path), exist_ok=True)

with open(save_path, "w") as f:
    json.dump(metrics, f, indent=4)

print(f"📄 Metrics and prediction records saved to: {save_path}")


/data_ft_8/keru/conda/envs/yolov5/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/data_ft_8/keru/conda/envs/yolov5/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
100%|██████████| 20/20 [00:01<00:00, 16.93it/s]

✅ Grad-CAM saved to /data_lg/keru/project/part2/DermNet/pseudo_eval/gradcam_pseudo
Accuracy: 0.5000
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
AUROC: 0.1300
📄 Metrics and prediction records saved to: /data_lg/keru/project/part2/DermNet/pseudo_eval/pseudo_eval_metrics.json



/data_ft_8/keru/conda/envs/yolov5/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [11]:
import os
import torch
from torchvision import transforms
from torch import nn
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from tqdm import tqdm
from PIL import Image
from facenet_pytorch import InceptionResnetV1
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
import json

# === 配置 ===
pseudo_txt = "/data_lg/keru/project/part2/DermNet/pseudo_eval/pseudo_test_list.txt"
model_path = "/data_lg/keru/project/part2/outputs_v2_weight/best_facenet_v2.pt"  # ← 改为你 v2/v3 模型路径
gradcam_dir = "/data_lg/keru/project/part2/DermNet/pseudo_eval/gradcam_v2_weight"

os.makedirs(gradcam_dir, exist_ok=True)

# === 模型加载（处理 module.） ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = InceptionResnetV1(pretrained='vggface2', classify=True, num_classes=2)

state_dict = torch.load(model_path, map_location=device)
new_state_dict = {k.replace("module.", ""): v for k, v in state_dict.items()}
model.load_state_dict(new_state_dict)
model = model.to(device)
model.eval()

# === 预处理 ===
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# === 读取伪测试集列表 ===
samples, labels = [], []
with open(pseudo_txt) as f:
    for line in f:
        path, lbl = line.strip().split("\t")
        samples.append(path)
        labels.append(int(lbl))

# === Grad-CAM 设置 ===
target_layer = model.block8.branch1[-1]  # 最后卷积
cam = GradCAM(model=model, target_layers=[target_layer])

# === 推理 ===
all_preds, all_probs, json_records = [], [], []

for img_path, true_label in tqdm(zip(samples, labels), total=len(samples)):
    img = Image.open(img_path).convert("RGB")
    input_tensor = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(input_tensor)
        prob = torch.softmax(output, dim=1)[0, 1].item()
        pred = output.argmax(dim=1).item()

    all_preds.append(pred)
    all_probs.append(prob)

    # Grad-CAM
    grayscale_cam = cam(input_tensor=input_tensor, targets=[ClassifierOutputTarget(1)])[0, :]
    rgb_img = transform(img).permute(1, 2, 0).cpu().numpy()
    rgb_img = (rgb_img - rgb_img.min()) / (rgb_img.max() - rgb_img.min())
    cam_image = show_cam_on_image(rgb_img, grayscale_cam, use_rgb=True)

    out_name = os.path.basename(img_path).split(".")[0] + "_gradcam.jpg"
    out_path = os.path.join(gradcam_dir, out_name)
    Image.fromarray(cam_image).save(out_path)

    json_records.append({
        "img": img_path,
        "true": int(true_label),
        "pred": int(pred),
        "prob": round(prob, 4),
        "correct": int(pred == int(true_label))
    })

print(f"\n✅ Grad-CAM saved to: {gradcam_dir}")

# === 计算指标 ===
acc = accuracy_score(labels, all_preds)
prec = precision_score(labels, all_preds)
rec = recall_score(labels, all_preds)
f1 = f1_score(labels, all_preds)
try:
    auc = roc_auc_score(labels, all_probs)
except:
    auc = float('nan')

print("\n🎯 Evaluation Metrics:")
print(f"Accuracy:  {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall:    {rec:.4f}")
print(f"F1 Score:  {f1:.4f}")
print(f"AUROC:     {auc:.4f}")

# === 保存 JSON 结果 ===
eval_dir = "/data_lg/keru/project/part2/DermNet/pseudo_eval"
os.makedirs(eval_dir, exist_ok=True)

metrics_path = os.path.join(eval_dir, "pseudo_eval_metrics.json")
records_path = os.path.join(eval_dir, "pseudo_eval_records.json")

with open(metrics_path, "w") as f:
    json.dump({
        "Accuracy": round(acc, 4),
        "Precision": round(prec, 4),
        "Recall": round(rec, 4),
        "F1 Score": round(f1, 4),
        "AUROC": round(auc, 4)
    }, f, indent=4)

with open(records_path, "w") as f:
    json.dump(json_records, f, indent=4)

print(f"\n📊 Metrics saved to: {metrics_path}")
print(f"📝 Per-image results saved to: {records_path}")


100%|██████████| 20/20 [00:01<00:00, 13.89it/s]


✅ Grad-CAM saved to: /data_lg/keru/project/part2/DermNet/pseudo_eval/gradcam_v2_weight

🎯 Evaluation Metrics:
Accuracy:  0.5000
Precision: 0.0000
Recall:    0.0000
F1 Score:  0.0000
AUROC:     0.4400

📊 Metrics saved to: /data_lg/keru/project/part2/DermNet/pseudo_eval/pseudo_eval_metrics.json
📝 Per-image results saved to: /data_lg/keru/project/part2/DermNet/pseudo_eval/pseudo_eval_records.json



/data_ft_8/keru/conda/envs/yolov5/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# FINAL TEST!!

## real ratio

In [13]:
import os

# === 配置 ===
train_root = "/data_lg/keru/project/part2/DermNet/test"
output_txt = "/data_lg/keru/project/part2/DermNet/eval/final_test_list.txt"

# === 找到所有子文件夹 ===
all_classes = [
    d for d in os.listdir(train_root)
    if os.path.isdir(os.path.join(train_root, d))
]

# Acne 类和 Non-Acne 类分开
acne_folders = [d for d in all_classes if "acne" in d.lower()]
non_acne_folders = [d for d in all_classes if d not in acne_folders]

assert acne_folders, "没有找到包含 'acne' 的文件夹！"

# === 收集所有 Acne 样本 ===
acne_samples = []
for folder in acne_folders:
    folder_path = os.path.join(train_root, folder)
    imgs = [os.path.join(folder_path, f) for f in os.listdir(folder_path)
            if f.lower().endswith((".jpg", ".jpeg", ".png"))]
    acne_samples.extend(imgs)

# === 收集所有 Non-Acne 样本 ===
non_acne_samples = []
for folder in non_acne_folders:
    folder_path = os.path.join(train_root, folder)
    imgs = [os.path.join(folder_path, f) for f in os.listdir(folder_path)
            if f.lower().endswith((".jpg", ".jpeg", ".png"))]
    non_acne_samples.extend(imgs)

# === 合并 & 保存 txt ===
with open(output_txt, "w") as f:
    for p in acne_samples:
        f.write(f"{p}\t1\n")
    for p in non_acne_samples:
        f.write(f"{p}\t0\n")

print(f"Pseudo test list saved to {output_txt}")
print(f"Acne: {len(acne_samples)}, Non-Acne: {len(non_acne_samples)}")

Pseudo test list saved to /data_lg/keru/project/part2/DermNet/eval/final_test_list.txt
Acne: 312, Non-Acne: 3690


In [2]:
import os
import torch
from torchvision import transforms
from torch import nn
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from tqdm import tqdm
from PIL import Image
from facenet_pytorch import InceptionResnetV1
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
import json
import random

# === 配置 ===
# pseudo_txt = "/data_lg/keru/project/part2/DermNet/eval/final_test_list.txt" #真实比例
pseudo_txt = "/data_lg/keru/project/part2/DermNet/eval_real/final_test_list.txt"
model_path = "/data_lg/keru/project/part2/output/output_cycleGAN/best_facenet_v2.pt"  # ← 改为你 v2/v3 模型路径
gradcam_dir = "/data_lg/keru/project/part2/DermNet/eval_real/eval_cyclegan"
eval_dir = "/data_lg/keru/project/part2/DermNet/eval_real/eval_cyclegan"
os.makedirs(eval_dir, exist_ok=True)
os.makedirs(gradcam_dir, exist_ok=True)

# === 模型加载（处理 module.） ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = InceptionResnetV1(pretrained='vggface2', classify=True, num_classes=2)

state_dict = torch.load(model_path, map_location=device)
new_state_dict = {k.replace("module.", ""): v for k, v in state_dict.items()}
model.load_state_dict(new_state_dict)
model = model.to(device)
model.eval()

# === 预处理 ===
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# === 读取伪测试集列表 ===
samples, labels = [], []
with open(pseudo_txt) as f:
    for line in f:
        path, lbl = line.strip().split("\t")
        samples.append(path)
        labels.append(int(lbl))

# === Grad-CAM 设置 ===
target_layer = model.block8.branch1[-1]  # 最后卷积
cam = GradCAM(model=model, target_layers=[target_layer])

# === 推理 ===
all_preds, all_probs, json_records = [], [], []
correct_samples = []
wrong_samples = []

for img_path, true_label in tqdm(zip(samples, labels), total=len(samples)):
    img = Image.open(img_path).convert("RGB")
    input_tensor = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(input_tensor)
        prob = torch.softmax(output, dim=1)[0, 1].item()
        pred = output.argmax(dim=1).item()

    all_preds.append(pred)
    all_probs.append(prob)
    is_correct = (pred == true_label)
    
    # 保存样本路径、预测结果和真实标签
    sample_info = {
        "img_path": img_path,
        "true_label": true_label,
        "pred": pred,
        "prob": prob,
        "is_correct": is_correct
    }
    
    if is_correct:
        correct_samples.append(sample_info)
    else:
        wrong_samples.append(sample_info)

    json_records.append({
        "img": img_path,
        "true": int(true_label),
        "pred": int(pred),
        "prob": round(prob, 4),
        "correct": int(is_correct)
    })

# === 随机选择7张正确和7张错误的样本 ===
selected_correct = random.sample(correct_samples, min(7, len(correct_samples)))
selected_wrong = random.sample(wrong_samples, min(7, len(wrong_samples)))
selected_samples = selected_correct + selected_wrong

# === 为选中的样本生成并保存Grad-CAM ===
for i, sample in enumerate(selected_samples):
    img = Image.open(sample["img_path"]).convert("RGB")
    input_tensor = transform(img).unsqueeze(0).to(device)
    
    # 生成Grad-CAM
    grayscale_cam = cam(input_tensor=input_tensor, targets=[ClassifierOutputTarget(1)])[0, :]
    rgb_img = transform(img).permute(1, 2, 0).cpu().numpy()
    rgb_img = (rgb_img - rgb_img.min()) / (rgb_img.max() - rgb_img.min())
    cam_image = show_cam_on_image(rgb_img, grayscale_cam, use_rgb=True)
    
    # 确定输出文件名和路径
    result_type = "correct" if sample["is_correct"] else "wrong"
    img_name = os.path.basename(sample["img_path"]).split(".")[0]
    out_name = f"{result_type}_{i+1}_{img_name}_gradcam.jpg"
    out_path = os.path.join(gradcam_dir, out_name)
    
    # 保存Grad-CAM图像
    Image.fromarray(cam_image).save(out_path)

print(f"\n✅ Grad-CAM saved to: {gradcam_dir}")
print(f"   正确样本: {len(selected_correct)} 张")
print(f"   错误样本: {len(selected_wrong)} 张")

# === 计算指标 ===
acc = accuracy_score(labels, all_preds)
prec = precision_score(labels, all_preds)
rec = recall_score(labels, all_preds)
f1 = f1_score(labels, all_preds)
try:
    auc = roc_auc_score(labels, all_probs)
except:
    auc = float('nan')

print("\n🎯 Evaluation Metrics:")
print(f"Accuracy:  {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall:    {rec:.4f}")
print(f"F1 Score:  {f1:.4f}")
print(f"AUROC:     {auc:.4f}")

# === 保存 JSON 结果 ===


metrics_path = os.path.join(eval_dir, "eval_metrics.json")
records_path = os.path.join(eval_dir, "eval_records.json")

with open(metrics_path, "w") as f:
    json.dump({
        "Accuracy": round(acc, 4),
        "Precision": round(prec, 4),
        "Recall": round(rec, 4),
        "F1 Score": round(f1, 4),
        "AUROC": round(auc, 4)
    }, f, indent=4)

with open(records_path, "w") as f:
    json.dump(json_records, f, indent=4)

print(f"\n📊 Metrics saved to: {metrics_path}")
print(f"📝 Per-image results saved to: {records_path}")

100%|██████████| 4002/4002 [01:15<00:00, 53.35it/s]



✅ Grad-CAM saved to: /data_lg/keru/project/part2/DermNet/eval_real/eval_cyclegan
   正确样本: 7 张
   错误样本: 7 张

🎯 Evaluation Metrics:
Accuracy:  0.6794
Precision: 0.1100
Recall:    0.4391
F1 Score:  0.1760
AUROC:     0.6206

📊 Metrics saved to: /data_lg/keru/project/part2/DermNet/eval_real/eval_cyclegan/eval_metrics.json
📝 Per-image results saved to: /data_lg/keru/project/part2/DermNet/eval_real/eval_cyclegan/eval_records.json


## initial 版本评估

In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from tqdm import tqdm  # 使用普通tqdm，仅文本进度条
from PIL import Image
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
import json
import random
import matplotlib.pyplot as plt  # 仅用于保存图表，不显示


def build_model(name):
    """构建指定类型的模型"""
    if name == "resnet18":
        model = models.resnet18(pretrained=True)
        model.fc = nn.Linear(model.fc.in_features, 2)
    elif name == "resnet50":
        model = models.resnet50(pretrained=True)
        model.fc = nn.Linear(model.fc.in_features, 2)
    elif name == "efficientnet_b0":
        model = models.efficientnet_b0(pretrained=True)
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, 2)
    else:
        raise ValueError(f"Unsupported model: {name}")
    return model


def main():
    """模型验证主函数（纯命令行版本）"""
    # === 配置参数（需手动修改路径）===
    pseudo_txt = "/data_lg/keru/project/part2/DermNet/eval/final_test_list.txt"  # txt测试集路径
    model_path = "/data_lg/keru/project/part2/outputs/best_resnet50.pt"  # 模型路径
    gradcam_dir = "/data_lg/keru/project/part2/DermNet/eval/eval_initial/gram"  # Grad-CAM保存目录
    eval_dir = "/data_lg/keru/project/part2/DermNet/eval/eval_initial"  # 评估结果保存目录
    batch_size = 32
    model_name = "resnet50"

    os.makedirs(gradcam_dir, exist_ok=True)
    os.makedirs(eval_dir, exist_ok=True)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"🔥 使用设备: {device}")

    # === 模型加载 ===
    model = build_model(model_name).to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    print(f"✅ 模型加载自: {model_path}")

    # === 数据预处理 ===
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
    ])

    # === 从txt读取测试集 ===
    samples, labels = [], []
    with open(pseudo_txt) as f:
        for line in f:
            path, lbl = line.strip().split("\t")
            samples.append(path)
            labels.append(int(lbl))
    print(f"📊 加载 {len(samples)} 个测试样本")

    # === Grad-CAM设置 ===
    target_layer = model.layer4[-1] if 'resnet' in model_name else model.features[-1]
    cam = GradCAM(model=model, target_layers=[target_layer])

    # === 推理并收集结果 ===
    all_preds, all_probs, json_records = [], [], []
    correct_samples, wrong_samples = [], []

    model.eval()
    with torch.no_grad():
        pbar = tqdm(enumerate(zip(samples, labels)), total=len(samples), desc="推理中")
        for i, (img_path, true_label) in pbar:
            img = Image.open(img_path).convert("RGB")
            input_tensor = transform(img).unsqueeze(0).to(device)

            outputs = model(input_tensor)
            prob = torch.softmax(outputs, dim=1)[:, 1].item()
            pred = torch.argmax(outputs, dim=1).item()

            all_preds.append(pred)
            all_probs.append(prob)
            is_correct = (pred == true_label)

            # 保存样本信息
            sample_info = {
                "img_path": img_path,
                "true_label": true_label,
                "pred": pred,
                "prob": prob,
                "is_correct": is_correct
            }

            if is_correct:
                correct_samples.append(sample_info)
            else:
                wrong_samples.append(sample_info)

            json_records.append({
                "img": img_path,
                "true": int(true_label),
                "pred": int(pred),
                "prob": round(prob, 4),
                "correct": int(is_correct)
            })

    # === 随机选择7张正确和7张错误样本 ===
    selected_correct = random.sample(correct_samples, min(7, len(correct_samples)))
    selected_wrong = random.sample(wrong_samples, min(7, len(wrong_samples)))
    selected_samples = selected_correct + selected_wrong
    print(f"🔍 选中 {len(selected_correct)} 正确样本和 {len(selected_wrong)} 错误样本生成Grad-CAM")

    # === 生成并保存Grad-CAM ===
    gradcam_results = []
    for i, sample in enumerate(selected_samples):
        img = Image.open(sample["img_path"]).convert("RGB")
        input_tensor = transform(img).unsqueeze(0).to(device)

        # 生成Grad-CAM
        target_class = sample["pred"]
        targets = [ClassifierOutputTarget(target_class)]
        grayscale_cam = cam(input_tensor=input_tensor, targets=targets)[0, :]

        rgb_image = input_tensor.squeeze(0).permute(1, 2, 0).cpu().numpy()
        rgb_image = (rgb_image - rgb_image.min()) / (rgb_image.max() - rgb_image.min())
        visualization = show_cam_on_image(rgb_image, grayscale_cam, use_rgb=True)

        # 保存图像
        result_type = "correct" if sample["is_correct"] else "wrong"
        img_name = os.path.basename(sample["img_path"]).split(".")[0]
        out_name = f"{result_type}_{i + 1}_{img_name}_gradcam.jpg"
        out_path = os.path.join(gradcam_dir, out_name)
        Image.fromarray(visualization).save(out_path)

        # 记录Grad-CAM信息
        gradcam_results.append({
            "image_path": sample["img_path"],
            "true_label": sample["true_label"],
            "predicted_class": target_class,
            "probability": sample["prob"],
            "gradcam_path": out_path,
            "is_correct": sample["is_correct"]
        })

        # 仅文本提示，不显示图像
        print(f"\n📸 已保存 {result_type} 样本 {i+1} 的Grad-CAM: {out_name}")

    # === 计算评估指标 ===
    acc = accuracy_score(labels, all_preds)
    prec = precision_score(labels, all_preds)
    rec = recall_score(labels, all_preds)
    f1 = f1_score(labels, all_preds)
    try:
        auc = roc_auc_score(labels, all_probs)
    except:
        auc = float('nan')

    print(f"\n🎯 评估指标:")
    print(f"  准确率: {acc:.4f}")
    print(f"  精确率: {prec:.4f}")
    print(f"  召回率: {rec:.4f}")
    print(f"  F1分数: {f1:.4f}")
    print(f"  AUROC: {auc:.4f}")

    # === 保存JSON结果 ===
    metrics_path = os.path.join(eval_dir, "eval_metrics.json")
    records_path = os.path.join(eval_dir, "eval_records.json")
    gradcam_path = os.path.join(eval_dir, "gradcam_summary.json")

    with open(metrics_path, "w") as f:
        json.dump({
            "Accuracy": round(acc, 4),
            "Precision": round(prec, 4),
            "Recall": round(rec, 4),
            "F1 Score": round(f1, 4),
            "AUROC": round(auc, 4)
        }, f, indent=4)

    with open(records_path, "w") as f:
        json.dump(json_records, f, indent=4)

    with open(gradcam_path, "w") as f:
        json.dump(gradcam_results, f, indent=4)

    print(f"\n📊 指标保存至: {metrics_path}")
    print(f"📝 样本记录保存至: {records_path}")
    print(f"🔥 Grad-CAM汇总保存至: {gradcam_path}")

    # === 生成指标可视化图表（仅保存，不显示）===
    plt.figure(figsize=(10, 6))
    metrics = [acc, prec, rec, f1, auc]
    labels = ["Accuracy", "Precision", "Recall", "F1 Score", "AUROC"]
    plt.bar(labels, metrics, color=['skyblue', 'lightgreen', 'lightcoral', 'plum', 'lightyellow'])
    plt.ylim(0, 1.05)
    plt.title('模型评估指标')
    plt.ylabel('分数')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()

    metrics_plot = os.path.join(eval_dir, "metrics_barplot.png")
    plt.savefig(metrics_plot)
    plt.close()

    print(f"📈 指标图表已保存至: {metrics_plot}")
    print("（所有结果可在保存目录中查看）")


if __name__ == "__main__":
    main()

# 1:1 positive & negative

In [26]:
import os
import random

# === 配置 ===
train_root = "/data_lg/keru/project/part2/DermNet/test"
output_txt = "/data_lg/keru/project/part2/DermNet/eval/final_test_list_ratio.txt"
random_seed = 42  # 设置随机种子，保证结果可复现

# 设置随机种子
random.seed(random_seed)

# === 找到所有子文件夹 ===
all_classes = [
    d for d in os.listdir(train_root)
    if os.path.isdir(os.path.join(train_root, d))
]

# Acne 类和 Non-Acne 类分开
acne_folders = [d for d in all_classes if "acne" in d.lower()]
non_acne_folders = [d for d in all_classes if d not in acne_folders]

assert acne_folders, "没有找到包含 'acne' 的文件夹！"

# === 收集所有 Acne 样本 ===
acne_samples = []
for folder in acne_folders:
    folder_path = os.path.join(train_root, folder)
    imgs = [os.path.join(folder_path, f) for f in os.listdir(folder_path)
            if f.lower().endswith((".jpg", ".jpeg", ".png"))]
    acne_samples.extend(imgs)

# === 收集所有 Non-Acne 样本 ===
non_acne_samples = []
for folder in non_acne_folders:
    folder_path = os.path.join(train_root, folder)
    imgs = [os.path.join(folder_path, f) for f in os.listdir(folder_path)
            if f.lower().endswith((".jpg", ".jpeg", ".png"))]
    non_acne_samples.extend(imgs)

# === 随机选择与Acne样本数量相同的Non-Acne样本 ===
num_acne = len(acne_samples)
num_non_acne = len(non_acne_samples)

if num_non_acne >= num_acne:
    # 如果Non-Acne样本数量足够，随机选择与Acne相同数量的样本
    selected_non_acne = random.sample(non_acne_samples, num_acne)
else:
    # 如果Non-Acne样本数量不足，使用全部Non-Acne样本并对其进行重复采样
    print(f"警告: Non-Acne样本数量({num_non_acne})少于Acne样本数量({num_acne})，将进行重复采样")
    selected_non_acne = random.choices(non_acne_samples, k=num_acne)

# === 合并 & 保存 txt ===
with open(output_txt, "w") as f:
    # 写入Acne样本
    for p in acne_samples:
        f.write(f"{p}\t1\n")
    # 写入随机选择的Non-Acne样本
    for p in selected_non_acne:
        f.write(f"{p}\t0\n")

print(f"Pseudo test list saved to {output_txt}")
print(f"Acne: {len(acne_samples)}, Non-Acne: {len(selected_non_acne)}")
print(f"正负样本比例: {len(acne_samples)}:{len(selected_non_acne)} = 1:1")

Pseudo test list saved to /data_lg/keru/project/part2/DermNet/eval/final_test_list_ratio.txt
Acne: 312, Non-Acne: 312
正负样本比例: 312:312 = 1:1
